In [300]:
###################################################
### Publication or Commercial usage not allowed ###
###          v.rafanavicius@gmail.com           ###
###  If you are writing publication using this  ###
###             code please contact             ###
###################################################
%matplotlib inline
%run functions.py
from retrying import retry
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import folium
import csv
import pandas as pd
import simplekml
import math
from noaa_sdk import noaa
import requests
from datetime import datetime, timedelta
from netCDF4 import num2date
from siphon.catalog import TDSCatalog
from metpy.units import units
import metpy.calc as mc
import geopy
from geopy import Point
from geopy import distance
import random
import warnings
from folium.plugins import HeatMap
warnings.filterwarnings('ignore')
from bokeh.io import push_notebook, output_notebook, show
from bokeh.plotting import figure
output_notebook()
#TODO: Document the code
#TODO: Speed-up by saving intermediate results
#TODO: Baloon landing site prediction
#TODO: Implement static class instead of inheritance
#TODO: Launch site prediction from landing site (reverse the wind and altitude path)
#TODO: Test with real data
#TODO: Visualization directly in the map + Graphs
#TODO: Write README

Loading BokehJS ...

In [298]:
class Atmosphere(object): 
    def pres2alt(self,pressure):
        '''
        Determine altitude from site pressure.

        Parameters
        ----------
        pressure : numeric
            Atmospheric pressure (Pascals)

        Returns
        -------
        altitude : numeric
            Altitude in meters above sea level

        Notes
        ------
        The following assumptions are made

        ============================   ================
        Parameter                      Value
        ============================   ================
        Base pressure                  101325 Pa
        Temperature at zero altitude   288.15 K
        Gravitational acceleration     9.80665 m/s^2
        Lapse rate                     -6.5E-3 K/m
        Gas constant for air           287.053 J/(kgK)
        Relative Humidity              0%
        ============================   ================

        References
        -----------
        .. [1] "A Quick Derivation relating altitude to air pressure" from
           Portland State Aerospace Society, Version 1.03, 12/22/2004.
        '''

        alt = 44331.5 - 4946.62 * pressure ** (0.190263)

        return alt



    def alt2pres(self,altitude):
        '''
        Determine site pressure from altitude.

        Parameters
        ----------
        altitude : numeric
            Altitude in meters above sea level

        Returns
        -------
        pressure : numeric
            Atmospheric pressure (Pascals)

        Notes
        ------
        The following assumptions are made

        ============================   ================
        Parameter                      Value
        ============================   ================
        Base pressure                  101325 Pa
        Temperature at zero altitude   288.15 K
        Gravitational acceleration     9.80665 m/s^2
        Lapse rate                     -6.5E-3 K/m
        Gas constant for air           287.053 J/(kgK)
        Relative Humidity              0%
        ============================   ================

        References
        -----------
        .. [1] "A Quick Derivation relating altitude to air pressure" from
           Portland State Aerospace Society, Version 1.03, 12/22/2004.
        '''

        press = 100 * ((44331.514 - altitude) / 11880.516) ** (1 / 0.1902632)

        return press

class Wind: 
    def __init__(self): 
        self.position = np.array([23,54], dtype=np.float32) 
        self.speed = 0 
        self.direction = 0 
        self.pressure = 100000 * units('Pa') 
        self.names = ['u_wind, m/s', 'v_wind, m/s', 'pressure, Pa'] 
        self.rho = 1.225 * units('kg')/units('cubic meter')
       
  # Copyright (c) 2013-2015 Siphon Contributors. 
  # Distributed under the terms of the BSD 3-Clause License. 
    @retry
    def get_wind_nccs(self):
        '''
        Get wind components from nccs at different altitudes.

        Parameters
        ----------

        Returns
        -------
        u_wind, v_wind, pressure : netCDF4.Variable
        '''
        position = self.position 
        ########################################### 
        # First we construct a TDSCatalog instance pointing to our dataset of interest, in 
        # this case TDS' "Best" virtual dataset for the GFS global 0.25 degree collection of 
        # GRIB files. We see this catalog contains a single dataset. 

        #definetly needs some error management!!! 
        best_gfs = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/' 
                    'Global_0p25deg/catalog.xml?dataset=grib/NCEP/GFS/Global_0p25deg/Best') 
        ########################################### 
        # We pull out this dataset and get the NCSS access point 
        best_ds = best_gfs.datasets[0] 
        ncss = best_ds.subset() 
        ########################################### 
        # We can then use the `ncss` object to create a new query object, which 
        # facilitates asking for data from the server. 
        query = ncss.query() 
        ########################################### 
        # We construct a query asking for data
        now = datetime.utcnow() 
        query.lonlat_point(position[0], position[1]).time(datetime.utcnow()) 
#         query.lonlat_point(position[0], position[1]).time(datetime.fromisoformat('2011-11-04 00:05:23.283'))
        
        ########################################### 
        # We now request data from the server using this query. The `NCSS` class handles parsing
        query.variables('u-component_of_wind_isobaric', 'v-component_of_wind_isobaric') 
        query.accept('netcdf') 
        data = ncss.get_data(query) 
        query = ncss.query() 
        u_wind = data.variables['u-component_of_wind_isobaric'] 
        v_wind = data.variables['v-component_of_wind_isobaric'] 
        pressure = data.variables['isobaric'] 
        #       TODO: SAVE TO DF AND LOAD UNTIL 0p25deg CHANGE
        return u_wind, v_wind, pressure 
    
    def get_wind_data(self, array): 
        nparray = np.array(0) 
        df = pd.DataFrame() 
        for idx in array: 
            nparray = idx.flatten() 
        return nparray.astype(float) 

    def get_wind_df(self): 
        parameters = self.get_wind_nccs() 
        names = ['u_wind, m/s', 'v_wind, m/s', 'pressure, Pa'] 
        df = pd.DataFrame() 
        for name, parameter in zip(names, parameters): 
            df[name] = self.get_wind_data(parameter) 
        return df  
   
    def get_wind_full_df(self): 
        df = self.get_wind_df() 
        directions = [] 
        speeds = [] 
        for ix in range(len(df.index)): 
            u_wind2 = df['u_wind, m/s'].iloc[ix]*units('meter')/units('second') 
            v_wind2 = df['v_wind, m/s'].iloc[ix]*units('meter')/units('second') 
            direction = mc.wind_direction(u_wind2,v_wind2, convention='to') 
            speed = mc.wind_speed(u_wind2,v_wind2) 
            directions.append(direction) 
            speeds.append(speed) 
        df['direction'] = directions 
        df['speed'] = speeds 
        return df 
   
    def save_df(self): 
        df = self.get_wind_full_df() 
        df.to_pickle("weather.pkl") 
       
    def load_df(self): 
        return pd.read_pickle("weather.pkl") 

    def get_wind_interpolate(self, name, load_df=False): 
        value = self.pressure 
        if load_df==False: 
            df = self.get_wind_full_df() 
        else: 
            df = self.load_df() 
      #interpolation function given any pressure value return interpolated wind speed or direction 
        exactmatch=df[df['pressure, Pa']==value] 
        if value > 100000.0: 
            return df[df['pressure, Pa']==100000][name].values.squeeze() 
        elif (exactmatch.empty == False): 
            return exactmatch[name].values.squeeze() 
        else: 
            df1 = df[df['pressure, Pa']<value] 
            df2 = df[df['pressure, Pa']>value] 
            df_min = df1[df1['pressure, Pa'] == df1['pressure, Pa'].max()] 
            df_max = df2[df2['pressure, Pa'] == df2['pressure, Pa'].min()] 
            x_min = df_min['pressure, Pa'].values 
            x_max = df_max['pressure, Pa'].values 
            y_min = (df_min[name]).values 
            y_max = (df_max[name]).values 
            dy = y_max - y_min 
            dx = x_max - x_min 
            A = y_min - (dy/dx) * x_min 
            B = dy/dx 
            y_int = A + B * value 
            return y_int.squeeze() 
       
    def get_wind_direction(self): 
        return self.get_wind_interpolate('direction') 
   
    def get_wind_speed(self): 
        return self.get_wind_interpolate('speed') 
   
     
class Baloon(Wind): 
    def __init__(self, time_step = 600): 
        self.id = 0 
        self.position = np.array([11.89705, 49.42436], dtype=np.float32) 
        self.speed = 0  * units('meters')/units('second') 
        self.ascent_rate = 5 * units('meters')/units('second') # m/s 
        self.descent_rate = -7 * units('meters')/units('second')
        self.heading = 0 * units('degree') 
        self.altitude = 0 * units('meter') 
        self.burst_altitude = 30000 * units('meter')
        self.prev_a = 0   
        self.time_step = units.Quantity(time_step, 'second')
        self.mass = 1 * units('kilogram')
        self.Cd = 0.75
        self.Ap = 3 * units('square meter')
   
    def get_coord_float(self, lat_lon): 
        return [float(idx) for idx in lat_lon.split(',')] 
       
  # Physics function here 
    def get_baloon_coords(self): 
        wind_position = self.position 
        wind_speed = wind.get_wind_speed() 
        direction = self.get_baloon_direction() 
        horizontal_shift = self.get_baloon_dist() 
        direction = np.rad2deg(direction.astype(float)) 
        lat_lon = geopy.distance.geodesic(kilometers=horizontal_shift.magnitude).destination(Point(self.position[1],self.position[0
        ]), direction).format_decimal()
#       TODO: SAVE TO LOG FILE FUNCTION
#         print('dir: ',direction) 
#         print('wind spd: ',wind_speed) 
#         print('h shift: ',horizontal_shift) 
#         print('h shift: ',self.position) 
#         print(lat_lon) 
        x = self.get_coord_float(lat_lon) 
        return x 

  #Calculates horizontal distance of the baloon in one time step 
    def get_baloon_dist(self): 
        wind.position = self.position 
        wind_speed = wind.get_wind_speed() 
        return (wind_speed*units('meter')/units('second')*self.time_step).to('kilometer').astype(float) 
   
    def get_baloon_direction(self): 
        wind.position = self.position 
        return wind.get_wind_direction() 

    def get_baloon_elevation(self): 
        return self.ascent_rate*units('meter')/units('second')*self.time_step         
    
    def get_terminal_velocity(self):
        m = self.mass
        g = 9.87 * units('meter')/(units('second')*units('second'))
        p = wind.pressure
        Cd = self.Cd
        Ap = self.Ap

        h = self.altitude

        M = 0.0289654 * units('kilogram')/units('mole')
        R = 8.31447 * units('J')/(units('mole')*units('K'))
        L = 0.0065 * units('K')/units('meter')
        T0 = 288.15 * units('K')

        rho = round(p*M/(R*T0)*(1-pow((L*h)/T0,(g*M)/(R*L)-1)),3)
        Vt = -np.sqrt(2*m*g/(Cd*rho*Ap))
        Vter = Vt.magnitude*units('meter')/units('second')
#         print(Vter)
        return Vter

atm = Atmosphere() 
wind = Wind()     
ball = Baloon() 

In [252]:
df = wind.get_wind_full_df()

In [299]:
x = []
y = []
z = []
for index, row in df.iterrows():
    x = np.append(x, atm.pres2alt(df['pressure, Pa']))
    yy = np.append(y, df['speed'])
    zz = np.append(z, df['direction'])
for iy in range(0, len(yy)):
    y = np.append(y, yy[iy].magnitude)
    z = np.append(z, np.deg2rad(zz[iy].magnitude))

plot_scatter(x,y, xlabel='Aukštis virš jūros lygio, m', ylabel='Greitis, m/s', w=450, h=340, r=300)

plot_scatter(x,z, xlabel='Aukštis virš jūros lygio, m', ylabel='Kryptis, rad',w=450, h=340, r=300)


Show terminal velocity from model

In [ ]:
velocity = []
count = []
for x in range(0, 35000):
    ball.altitude = x * units('meter')
    wind.pressure = atm.alt2pres(ball.altitude.magnitude-4000) 
    velocity = np.append(velocity, ball.get_terminal_velocity().magnitude)
    count = np.append(count, x)

df_flight = pd.read_csv('Flights/F4DIW-14_edit.csv').iloc[250:]

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(tools=TOOLS)
p = figure(tools=TOOLS, plot_width=720, plot_height=480)
p.line(count[0:33000], velocity[0:33000], #radius=150, fill_alpha=1,
          color='red',line_color='red', line_width=4, legend='Aeolus modelis')

p.line([0,33000], [-10,-10], #radius=150, fill_alpha=1,
          color='red',line_color='blue', line_width=4, legend='CUSF 2.5 modelis')

p.scatter(df_flight['altitude'], df_flight['Clb'], radius=220, fill_alpha=1,
          color='black',line_color=None, line_width=4, legend="F4DIW-14 skrydis")

p.xaxis.axis_label = 'Aukštis virš jūros lygio, m'
p.yaxis.axis_label = 'Vertikalus greitis, m/s'
p.yaxis.axis_label_text_font_size = "14pt"
p.yaxis.major_label_text_font_size = "12pt"
p.xaxis.axis_label_text_font_size = "14pt"
p.xaxis.major_label_text_font_size = "12pt"
show(p)

Show changing integration intervals

In [341]:
altitude = []
altitude = np.linspace(0.0,30000.0, 31)
time_step = []
for alt in altitude:
    time_step = np.append(time_step, 0.1+1.033227e3*np.exp(-1*alt/8.188704e3))

plot_scatter(altitude, time_step, xlabel='Aukštis virš jūros lygio, m', ylabel='Integravimo žingnis, s', r=250, w=640, h=360)


In [3]:
###   PHYSICS   ###
#TODO: wrap in some function
def simulate(mass=7, Cd=0.4, Ap=1.0, 
             start_location = [11.89705, 49.42436], 
             start_altitude=0, burst_altitude=24700, 
             time_step=300, ascent_rate=7):
    #setup parameters,
    ball.mass = mass * units('kilogram')
    ball.Cd = Cd
    ball.Ap = Ap * units('square meter')
    start_location = start_location
    burst_altitude = burst_altitude * units('meter') # meters 
    ball.time_step = time_step * units('second') # seconds 
    ball.position = start_location # initial position
    ball.altitude = start_altitude * units('meter') 
    ball.ascent_rate = ascent_rate * units('meters')/units('second') 
    ball.burst_altitude = burst_altitude
    #ball.descent_rate = -3.2 * units('meters')/units('second')
#TODO: rename as array_mass etc.
    lat =[ball.position[1]] 
    lon = [ball.position[0]] 
    name = [0] 
    array_Vspeed = [0]
    Hspeed = [0]
    array_altitude = [ball.altitude.magnitude]
    array_step = [0]
    array_mass = [ball.mass.magnitude]
    array_Cd = [ball.Cd]
    array_Ap = [ball.Ap.magnitude]
    array_burst_altitude = [ball.burst_altitude.magnitude]
    print(ball.altitude)    

    #Ascending
    while(ball.altitude < burst_altitude): 
        wind.pressure = atm.alt2pres(ball.altitude.magnitude-4000) 
        coords = ball.get_baloon_coords() 
        ball.position = np.flip(coords) 
        name = np.append(name, ball.altitude.magnitude)
        elevation = ball.time_step*ball.ascent_rate
        ball.altitude += elevation    
        if ball.altitude > burst_altitude:
            ball.altitude = burst_altitude
            wind.pressure = atm.alt2pres(ball.altitude.magnitude-4000) 
            coords = ball.get_baloon_coords() 
            ball.position = np.flip(coords) 
            lat = np.append(lat,coords[0]) 
            lon = np.append(lon,coords[1])
            burst_coords = [coords[1], coords[0]]
            elevation = burst_altitude 
            name = np.append(name, burst_altitude.magnitude)
            array_Vspeed = np.append(array_Vspeed,ball.ascent_rate.magnitude)
            array_altitude = np.append(array_altitude, ball.altitude.magnitude)
            array_step = np.append(array_step, ball.time_step.magnitude)
            array_mass = np.append(array_mass, ball.mass.magnitude)
            array_Cd = np.append(array_Cd, ball.Cd)
            array_Ap = np.append(array_Ap, ball.Ap.magnitude)
            array_burst_altitude = np.append(array_burst_altitude, ball.burst_altitude.magnitude)
            print(ball.altitude)
        else:
            lat = np.append(lat,coords[0]) 
            lon = np.append(lon,coords[1]) 
            name = np.append(name, burst_altitude.magnitude)
            array_altitude = np.append(array_altitude, ball.altitude.magnitude)
            array_Vspeed = np.append(array_Vspeed,ball.ascent_rate.magnitude)
            array_step = np.append(array_step, ball.time_step.magnitude)
            array_mass = np.append(array_mass, ball.mass.magnitude)
            array_Cd = np.append(array_Cd, ball.Cd)
            array_Ap = np.append(array_Ap, ball.Ap.magnitude)
            array_burst_altitude = np.append(array_burst_altitude, ball.burst_altitude.magnitude)
            print(ball.altitude)

    #ball.time_step = 600*units('second') # seconds         
    #Descending    
    while(ball.altitude > 0):
        ball.time_step = 0.1+1.033227e3*np.exp(-ball.altitude.magnitude/8.188704e3)
        ball.time_step = ball.time_step*units('second')
        ball.descent_rate = round(ball.get_terminal_velocity(),2)
        elevation = ball.time_step*ball.descent_rate 
        ball.altitude += elevation
        if ball.altitude > 0:
            wind.pressure = atm.alt2pres(ball.altitude.magnitude-4000) 
            coords = ball.get_baloon_coords() 
            ball.position = np.flip(coords) 
            lat = np.append(lat,coords[0]) 
            lon = np.append(lon,coords[1]) 
            name = np.append(name, ball.altitude.magnitude)
            array_Vspeed = np.append(array_Vspeed,ball.descent_rate.magnitude)
            array_altitude = np.append(array_altitude, ball.altitude.magnitude)
            array_step = np.append(array_step, ball.time_step.magnitude)
            array_mass = np.append(array_mass, ball.mass.magnitude)
            array_Cd = np.append(array_Cd, ball.Cd)
            array_Ap = np.append(array_Ap, ball.Ap.magnitude)
            array_burst_altitude = np.append(array_burst_altitude, ball.burst_altitude.magnitude)
            print(round(ball.altitude,0))
        else:
            ball.altitude=0*units('meter')
            ball.time_step = 0.1+1.033227e3*np.exp(-ball.altitude.magnitude/8.188704e3)
            ball.time_step = ball.time_step*units('second')
            ball.descent_rate = round(ball.get_terminal_velocity(),2)
            wind.pressure = atm.alt2pres(ball.altitude.magnitude-4000) 
            coords = ball.get_baloon_coords() 
            ball.position = np.flip(coords) 
            lat = np.append(lat,coords[0]) 
            lon = np.append(lon,coords[1])  
            name = np.append(name, ball.altitude.magnitude)
            array_Vspeed = np.append(array_Vspeed,ball.descent_rate.magnitude)
            array_altitude = np.append(array_altitude, ball.altitude.magnitude)
            array_step = np.append(array_step, ball.time_step.magnitude)
            array_mass = np.append(array_mass, ball.mass.magnitude)
            array_Cd = np.append(array_Cd, ball.Cd)
            array_Ap = np.append(array_Ap, ball.Ap.magnitude)
            array_burst_altitude = np.append(array_burst_altitude, ball.burst_altitude.magnitude)
            print(ball.altitude)
            
    data = {
        'Lat': lat, 
        'Lon': lon, 
        'Altitude': array_altitude,
        'Vspeed': array_Vspeed,
        'mass': array_mass,
        'Cd': array_Cd,
        'Ap': array_Ap,
        'Burst altitude': array_burst_altitude,
        'Time_step': array_step
       } 
    df = pd.DataFrame(data=data)
    return df

In [4]:
# df = simulate(mass=4, Cd=0.5, Ap=1.0, 
#              start_location = [11.89705, 49.42436],
#              start_altitude=0, burst_altitude=24700, 
#              time_step=5000, ascent_rate=6.7)

In [29]:
# data = {'Name': name.astype('str'),
#         'Lat': lat, 'Lon': lon, 
#         'Altitude': name.astype('str'), 
#         'Description':name.astype('str') 
#        } 
# df = pd.DataFrame(data=data) 
df.to_csv('Predictions/P3421101.csv')
# kml = simplekml.Kml() 
# df.apply(lambda X: kml.newpoint(name=X["Name"], description=X["Description"], coords=[( X["Lon"],X["Lat"],X["Altitude"])]) ,axis=1) 
# kml.save(path = "data.kml")

In [353]:
df_flight = pd.read_csv('Flights/R3960637.csv')
f_lat = df_flight['lat']
f_lon = df_flight['lng']

df_other = pd.read_csv('OtherModels/R3960637.csv')
other_lat = df_other['lat'] 
other_lon = df_other['lon']

# df = pd.read_csv('Predictions/P3421101_1.csv')
# model_lat = df['Lat'] 
# model_lon = df['Lon']
# df_model6 = pd.read_csv('Predictions/R3960637_p6.csv')
# df_model2 = pd.read_csv('Predictions/R3960637_p0.csv')
# df_model3 = pd.read_csv('Predictions/R3960637_p3.csv')
list(df_flight)
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(tools=TOOLS)

p.scatter(df_flight['altitude'], df_flight['speed'], radius=150, fill_alpha=1,
          color='black',line_color=None)
p.xaxis.axis_label = 'Aukštis virš jūros lygio, m'
p.yaxis.axis_label = 'Horizontalus greitis, km/h'
show(p)
# plot_simple_scatter(df_flight['time'], df_flight['altitude'], x_label='', y_label='', title='', save='False')

In [104]:

burst = df_flight[df_flight['altitude'] == df_flight['altitude'].max()]

loc = [df_flight['lat'].iloc[0], df_flight['lng'].iloc[0]]
m = folium.Map(location=[burst['lat'], burst['lng']])
folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('stamenterrain').add_to(m)
folium.LayerControl().add_to(m)

folium.Marker(
    location=loc,
    popup='Start',
    icon=folium.Icon(icon='home')
).add_to(m)

folium.Marker(
    location=[burst['lat'], burst['lng']],
    popup='Burst '+str(burst['altitude'])+' m',
    icon=folium.Icon(icon='fire')
).add_to(m)

folium.Marker(
    location=[df_flight['lat'].iloc[-1],df_flight['lng'].iloc[-1]],
    popup='Land '+str(df_flight['lat'].iloc[-1]) + ' , ' + str(df_flight['lng'].iloc[-1]),
    icon=folium.Icon(icon='search')
).add_to(m)

# folium.PolyLine(np.dstack((df_model6['Lat'],df_model6['Lon'])), color="red", weight=3, opacity=0.9).add_to(m)
# folium.PolyLine(np.dstack((df_model2['Lat'],df_model2['Lon'])), color="green", weight=3, opacity=0.9).add_to(m)


# folium.PolyLine(np.dstack((model_lat,model_lon)), color="red", weight=3, opacity=0.9).add_to(m)
folium.PolyLine(np.dstack((f_lat,f_lon)), color="black", weight=3, opacity=0.9).add_to(m)
folium.PolyLine(np.dstack((other_lat,other_lon)), color="blue", weight=3, opacity=0.9).add_to(m)

m


In [397]:
df_flight = pd.read_csv('Flights/R3960637_edit.csv')
f_lat = df_flight['lat']
f_lon = df_flight['lng']

df_other = pd.read_csv('OtherModels/R3960637.csv')
other_lat = df_other['lat'] 
other_lon = df_other['lon']

df_other = pd.read_csv('Predictions/R3960637_p2.csv')
model_lat = df_other['Lat'] 
model_lon = df_other['Lon']

other_lat2 = (np.intersect1d(model_lat, other_lat))
other_lon2 = (np.intersect1d(model_lon, other_lon))



TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(tools=TOOLS)

p.scatter(other_lon, other_lat, radius=0.005, fill_alpha=1,
          color='black',line_color=None)
p.scatter(other_lon2, other_lat2, radius=0.005, fill_alpha=1,
          color='red',line_color=None)

p.xaxis.axis_label = 'Aukštis virš jūros lygio, m'
p.yaxis.axis_label = 'Vertikalus greitis, m/s'
p.yaxis.axis_label_text_font_size = "14pt"
p.yaxis.major_label_text_font_size = "12pt"
p.xaxis.axis_label_text_font_size = "14pt"
p.xaxis.major_label_text_font_size = "12pt"
show(p)


df_all.head()

,Unnamed: 0,Lat,Lon,Altitude,Vspeed,mass,Cd,Ap,Burst altitude,Time_step
0,0,49.424360,11.897050,0.0,0.00,1.93,1.08,1.43,26586.67,0.0
1,1,49.423189,11.893818,1689.0,5.63,1.93,1.08,1.43,26586.67,300.0
2,2,49.422018,11.890585,3378.0,5.63,1.93,1.08,1.43,26586.67,300.0
3,3,49.420847,11.887353,5067.0,5.63,1.93,1.08,1.43,26586.67,300.0
4,4,49.427099,11.871738,6756.0,5.63,1.93,1.08,1.43,26586.67,300.0


In [ ]:
count = 0

while(count<200):

    mass_min = 1.9
    mass_max = 2.1

    Cd_min = 0.75
    Cd_max = 1.25

    Ap_min = 1.4
    Ap_max = 1.6

    burst_altitude_min = 21700
    burst_altitude_max = 27700

    ascent_rate_min = 4.0
    ascent_rate_max = 6.0

    mass = round(random.uniform(mass_min, mass_max), 2)
    Cd = round(random.uniform(Cd_min, Cd_max), 2)
    Ap = round(random.uniform(Ap_min, Ap_max), 2)
    burst_altitude = round(random.uniform(burst_altitude_min, burst_altitude_max), 2)
    ascent_rate = round(random.uniform(ascent_rate_min, ascent_rate_max), 2)


    df = simulate(mass=mass, Cd=Cd, Ap=Ap, 
                 start_location = [11.89705, 49.42436], 
                 start_altitude=0, burst_altitude=burst_altitude, 
                 time_step=300, ascent_rate=ascent_rate)

    df.to_csv('Predictions/P3421101_'+str(count)+'.csv')

    count +=1
    print(count)
print('finish')

Stochastic

In [409]:
import os
import glob
import openpyxl

def read_all_folder(path, tail):
    all_files = glob.glob(path)
    li = []
    for filename in all_files:
        if os.path.getsize(filename)>0:
            df = pd.read_csv(filename, header=0)
            if tail != 0:
                li.append(df.tail(tail))
            else:
                li.append(df)
        else:
            print("empty file: ", filename)
    print("reading files finished")
    frame = pd.concat(li,axis=0)
    return frame

MC_df0 = read_all_folder('Predictions/P34*', 1)
MC_df0.iloc[0:20].to_excel('P3421101_MC.xlsx')

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(tools=TOOLS)

p.scatter(MC_df0['mass'], MC_df0['Cd'], radius=0.003, fill_alpha=1,
          color='black',line_color=None)

p.xaxis.axis_label = 'Aukštis virš jūros lygio, m'
p.yaxis.axis_label = 'Vertikalus greitis, m/s'
p.yaxis.axis_label_text_font_size = "14pt"
p.yaxis.major_label_text_font_size = "12pt"
p.xaxis.axis_label_text_font_size = "14pt"
p.xaxis.major_label_text_font_size = "12pt"
show(p)

reading files finished


In [1]:

MC_df = MC_df0.iloc[0:22]

df_flight = pd.read_csv('Flights/P3421101_edit.csv')
f_lat = df_flight['lat']
f_lon = df_flight['lng']

df_model1 = pd.read_csv('Predictions/P3421101_0.csv')
m1_lat = df_model1['Lat']
m1_lon = df_model1['Lon']

df_model2 = pd.read_csv('Predictions/P3421101_1.csv')
m2_lat = df_model2['Lat']
m2_lon = df_model2['Lon']

df_model3 = pd.read_csv('Predictions/P3421101_2.csv')
m3_lat = df_model3['Lat']
m3_lon = df_model3['Lon']

df_model4 = pd.read_csv('Predictions/P3421101_3.csv')
m4_lat = df_model4['Lat']
m4_lon = df_model4['Lon']

df_model5 = pd.read_csv('Predictions/P3421101_4.csv')
m5_lat = df_model5['Lat']
m5_lon = df_model5['Lon']

df_model6 = pd.read_csv('Predictions/P3421101_5.csv')
m6_lat = df_model6['Lat']
m6_lon = df_model6['Lon']

df_model7 = pd.read_csv('Predictions/P3421101_6.csv')
m7_lat = df_model7['Lat']
m7_lon = df_model7['Lon']

# df_other = pd.read_csv('OtherModels/R3421101.csv')
# other_lat = df_other['lat'] 
# other_lon = df_other['lon']

burst = df_flight[df_flight['altitude'] == df_flight['altitude'].max()]

loc = [burst['lat'].iloc[0], burst['lng'].iloc[0]]
hm = folium.Map(location=loc)
folium.TileLayer('openstreetmap').add_to(hm)

folium.Marker(
    location=[df_flight['lat'].iloc[1], df_flight['lng'].iloc[1]],
    popup='Start',
    icon=folium.Icon(icon='home', color = 'black')
).add_to(hm)

folium.Marker(
    location=[burst['lat'], burst['lng']],
    popup='Burst '+str(burst['altitude'])+' m',
    icon=folium.Icon(icon='fire', color = 'black')
).add_to(hm)

folium.Marker(
    location=[df_flight['lat'].iloc[-1],df_flight['lng'].iloc[-1]],
    popup='Land '+str(df_flight['lat'].iloc[-1]) + ' , ' + str(df_flight['lng'].iloc[-1]),
    icon=folium.Icon(icon='search', color='black')
).add_to(hm)

# # folium.PolyLine(np.dstack((df_model6['Lat'],df_model6['Lon'])), color="red", weight=3, opacity=0.9).add_to(m)
# # folium.PolyLine(np.dstack((df_model2['Lat'],df_model2['Lon'])), color="green", weight=3, opacity=0.9).add_to(m)


folium.PolyLine(np.dstack((m1_lat,m1_lon)), color="red", weight=3, opacity=0.9).add_to(hm)
folium.PolyLine(np.dstack((m2_lat,m2_lon)), color="red", weight=3, opacity=0.9).add_to(hm)
folium.PolyLine(np.dstack((m3_lat,m3_lon)), color="red", weight=3, opacity=0.9).add_to(hm)
folium.PolyLine(np.dstack((m4_lat,m4_lon)), color="red", weight=3, opacity=0.9).add_to(hm)
folium.PolyLine(np.dstack((m5_lat,m5_lon)), color="red", weight=3, opacity=0.9).add_to(hm)
folium.PolyLine(np.dstack((m6_lat,m6_lon)), color="red", weight=3, opacity=0.9).add_to(hm)
folium.PolyLine(np.dstack((m7_lat,m7_lon)), color="red", weight=3, opacity=0.9).add_to(hm)
folium.PolyLine(np.dstack((f_lat,f_lon)), color="black", weight=3, opacity=0.9).add_to(hm)
# folium.PolyLine(np.dstack((other_lat,other_lon)), color="blue", weight=3, opacity=0.9).add_to(m)

hmap = HeatMap( list(zip(MC_df['Lat'], MC_df['Lon'], MC_df['Altitude'])),
                   min_opacity=0.3,
                   max_val=2000,
                   radius=50, blur=30, 
                   max_zoom=1, 
                 )

hm.add_child(hmap)
hm

NameError: name 'MC_df0' is not defined

In [278]:
df_flight = pd.read_csv('Flights/P3421101_edit.csv')


df_all = read_all_folder('Predictions/P3421101_3.csv', 0)

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(tools=TOOLS)

p.scatter(df_flight['altitude'], df_flight['Clb'], radius=150, fill_alpha=1,
          color='black',line_color=None)

p.scatter(df_all['Altitude'], df_all['Vspeed'], radius=250, fill_alpha=1,
          color='red',line_color=None)
p.xaxis.axis_label = 'Aukštis virš jūros lygio, m'
p.yaxis.axis_label = 'Vertikalus greitis, m/s'
p.yaxis.axis_label_text_font_size = "14pt"
p.yaxis.major_label_text_font_size = "12pt"
p.xaxis.axis_label_text_font_size = "14pt"
p.xaxis.major_label_text_font_size = "12pt"
show(p)


df_all.head()

reading files finished


,Unnamed: 0,Lat,Lon,Altitude,Vspeed,mass,Cd,Ap,Burst altitude,Time_step
0,0,49.424360,11.897050,0.0,0.00,1.93,1.08,1.43,26586.67,0.0
1,1,49.423189,11.893818,1689.0,5.63,1.93,1.08,1.43,26586.67,300.0
2,2,49.422018,11.890585,3378.0,5.63,1.93,1.08,1.43,26586.67,300.0
3,3,49.420847,11.887353,5067.0,5.63,1.93,1.08,1.43,26586.67,300.0
4,4,49.427099,11.871738,6756.0,5.63,1.93,1.08,1.43,26586.67,300.0


In [154]:
df_flight = pd.read_csv('Flights/P3421101_edit.csv')

df_flight = df_flight.iloc[0:958]
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(tools=TOOLS, plot_width=720, plot_height=480)

p.scatter(df_flight['altitude'], df_flight['Clb'], radius=150, fill_alpha=1,
          color='black',line_color=None)
p.line([0,25000],[5,5], color='red', line_width=5)
p.xaxis.axis_label = 'Aukštis virš jūros lygio, m'
p.yaxis.axis_label = 'Vertikalus greitis, m/s'
p.yaxis.axis_label_text_font_size = "14pt"
p.yaxis.major_label_text_font_size = "12pt"
p.xaxis.axis_label_text_font_size = "14pt"
p.xaxis.major_label_text_font_size = "12pt"
show(p)

In [135]:
%run functions.py
df_flight = pd.read_csv('Flights/F4DIW-14_edit.csv')
df_flight = df_flight.iloc[250:]
plot_scatter(df_flight['altitude'], df_flight['Clb'], 
             xlabel = 'aukštis virš jūros lygio, m', ylabel='vertikalus greitis, m/s',
             w=480, h=480, r=300)

df_flight = pd.read_csv('Flights/P3421101_edit.csv')
df_flight = df_flight.iloc[960:]
plot_scatter(df_flight['altitude'], df_flight['Clb'], 
             xlabel = 'aukštis virš jūros lygio, m', ylabel='vertikalus greitis, m/s',
             w=480, h=480, r=300)

df_flight = pd.read_csv('Flights/R0150226_edit.csv')
df_flight = df_flight.iloc[225:]
plot_scatter(df_flight['altitude'], df_flight['Clb'], 
             xlabel = 'aukštis virš jūros lygio, m', ylabel='vertikalus greitis, m/s',
             w=480, h=480, r=150)

df_flight = pd.read_csv('Flights/R3960637_edit.csv')
df_flight = df_flight.iloc[932:]
plot_scatter(df_flight['altitude'], df_flight['Clb'], 
             xlabel = 'aukštis virš jūros lygio, m', ylabel='vertikalus greitis, m/s',
             w=480, h=480, r=150)

In [102]:

from bokeh.plotting import figure, output_file, show

df_flight = pd.read_csv('Flights/R3960637_edit.csv')
f_lat = df_flight['lat']
f_lon = df_flight['lng']

df_other = pd.read_csv('OtherModels/R3960637.csv')
other_lat = df_other['lat'] 
other_lon = df_other['lon']

# df = pd.read_csv('Predictions/P3421101_1.csv')
# model_lat = df['Lat'] 
# model_lon = df['Lon']
list(df_flight)
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(tools=TOOLS)

p.scatter(df_flight['altitude'], df_flight['speed'], radius=150, fill_alpha=1,
          color='black',line_color=None)
p.xaxis.axis_label = 'Aukštis virš jūros lygio, m'
p.yaxis.axis_label = 'Horizontalus greitis, km/h'
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "11pt"
p.yaxis.major_label_text_font_size = "11pt"
show(p)

In [103]:
p2 = figure(tools=TOOLS)
p2.scatter(df_flight['altitude'], df_flight['Clb'], radius=150, fill_alpha=1,
          color='black',line_color=None)
p2.xaxis.axis_label = 'Aukštis virš jūros lygio, m'
p2.yaxis.axis_label = 'Vertikalus greitis, m/s'
p2.xaxis.axis_label_text_font_size = "16pt"
p2.yaxis.axis_label_text_font_size = "16pt"
p2.xaxis.major_label_text_font_size = "11pt"
p2.yaxis.major_label_text_font_size = "11pt"
show(p2)

In [116]:
from folium.plugins import HeatMap

burst = df_flight[df_flight['altitude'] == df_flight['altitude'].max()]
burst2 = df_other[df_other['altitude'] == df_other['altitude'].max()]

loc = [df_flight['lat'].iloc[0], df_flight['lng'].iloc[0]]
m = folium.Map(location=[burst['lat'], burst['lng']])
folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('stamenterrain').add_to(m)
folium.LayerControl().add_to(m)

folium.Marker(
    location=loc,
    popup='Start',
    icon=folium.Icon(icon='home')
).add_to(m)

folium.Marker(
    location=[burst['lat'], burst['lng']],
    popup='Burst '+str(burst['altitude'])+' m',
    icon=folium.Icon(icon='fire')
).add_to(m)

folium.Marker(
    location=[df_flight['lat'].iloc[-1],df_flight['lng'].iloc[-1]],
    popup='Land '+str(df_flight['lat'].iloc[-1]) + ' , ' + str(df_flight['lng'].iloc[-1]),
    icon=folium.Icon(icon='search')
).add_to(m)

folium.Marker(
    location=[burst2['lat'], burst2['lon']],
    popup='Burst '+str(burst2['altitude'])+' m',
    icon=folium.Icon(icon='fire')
).add_to(m)

folium.Marker(
    location=[df_other['lat'].iloc[-1],df_other['lon'].iloc[-1]],
    popup='Land '+str(df_other['lat'].iloc[-1]) + ' , ' + str(df_other['lon'].iloc[-1]),
    icon=folium.Icon(icon='search')
).add_to(m)

# folium.PolyLine(np.dstack((df_model6['Lat'],df_model6['Lon'])), color="red", weight=3, opacity=0.9).add_to(m)
# folium.PolyLine(np.dstack((df_model2['Lat'],df_model2['Lon'])), color="green", weight=3, opacity=0.9).add_to(m)


# folium.PolyLine(np.dstack((model_lat,model_lon)), color="red", weight=3, opacity=0.9).add_to(m)
folium.PolyLine(np.dstack((f_lat,f_lon)), color="black", weight=3, opacity=0.9).add_to(m)
folium.PolyLine(np.dstack((other_lat,other_lon)), color="blue", weight=3, opacity=0.9).add_to(m)

m

In [117]:
df_flight = pd.read_csv('Flights/R0150226_edit.csv')
f_lat = df_flight['lat']
f_lon = df_flight['lng']

df_other = pd.read_csv('OtherModels/R0150226.csv')
other_lat = df_other['lat'] 
other_lon = df_other['lon']

# df = pd.read_csv('Predictions/P3421101_1.csv')
# model_lat = df['Lat'] 
# model_lon = df['Lon']
list(df_flight)
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(tools=TOOLS)

p.scatter(df_flight['altitude'], df_flight['speed'], radius=150, fill_alpha=1,
          color='black',line_color=None)
p.xaxis.axis_label = 'Aukštis virš jūros lygio, m'
p.yaxis.axis_label = 'Horizontalus greitis, km/h'
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "11pt"
p.yaxis.major_label_text_font_size = "11pt"
show(p)

In [118]:
p2 = figure(tools=TOOLS)
p2.scatter(df_flight['altitude'], df_flight['Clb'], radius=150, fill_alpha=1,
          color='black',line_color=None)
p2.xaxis.axis_label = 'Aukštis virš jūros lygio, m'
p2.yaxis.axis_label = 'Vertikalus greitis, m/s'
p2.xaxis.axis_label_text_font_size = "16pt"
p2.yaxis.axis_label_text_font_size = "16pt"
p2.xaxis.major_label_text_font_size = "11pt"
p2.yaxis.major_label_text_font_size = "11pt"
show(p2)

In [119]:
from folium.plugins import HeatMap

burst = df_flight[df_flight['altitude'] == df_flight['altitude'].max()]
burst2 = df_other[df_other['altitude'] == df_other['altitude'].max()]

loc = [df_flight['lat'].iloc[0], df_flight['lng'].iloc[0]]
m = folium.Map(location=[burst['lat'], burst['lng']])
folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('stamenterrain').add_to(m)
folium.LayerControl().add_to(m)

folium.Marker(
    location=loc,
    popup='Start',
    icon=folium.Icon(icon='home')
).add_to(m)

folium.Marker(
    location=[burst['lat'], burst['lng']],
    popup='Burst '+str(burst['altitude'])+' m',
    icon=folium.Icon(icon='fire')
).add_to(m)

folium.Marker(
    location=[df_flight['lat'].iloc[-1],df_flight['lng'].iloc[-1]],
    popup='Land '+str(df_flight['lat'].iloc[-1]) + ' , ' + str(df_flight['lng'].iloc[-1]),
    icon=folium.Icon(icon='search')
).add_to(m)

folium.Marker(
    location=[burst2['lat'], burst2['lon']],
    popup='Burst '+str(burst2['altitude'])+' m',
    icon=folium.Icon(icon='fire')
).add_to(m)

folium.Marker(
    location=[df_other['lat'].iloc[-1],df_other['lon'].iloc[-1]],
    popup='Land '+str(df_other['lat'].iloc[-1]) + ' , ' + str(df_other['lon'].iloc[-1]),
    icon=folium.Icon(icon='search')
).add_to(m)

# folium.PolyLine(np.dstack((df_model6['Lat'],df_model6['Lon'])), color="red", weight=3, opacity=0.9).add_to(m)
# folium.PolyLine(np.dstack((df_model2['Lat'],df_model2['Lon'])), color="green", weight=3, opacity=0.9).add_to(m)


# folium.PolyLine(np.dstack((model_lat,model_lon)), color="red", weight=3, opacity=0.9).add_to(m)
folium.PolyLine(np.dstack((f_lat,f_lon)), color="black", weight=3, opacity=0.9).add_to(m)
folium.PolyLine(np.dstack((other_lat,other_lon)), color="blue", weight=3, opacity=0.9).add_to(m)

m

In [148]:
df_flight = pd.read_csv('Flights/R3960637_edit.csv')
f_lat = df_flight['lat']
f_lon = df_flight['lng']

df_other = pd.read_csv('OtherModels/R3960637.csv')
other_lat = df_other['lat'] 
other_lon = df_other['lon']

df_model = pd.read_csv('Predictions/R3960637_p2.csv')
model_lat = df_model['Lat'] 
model_lon = df_model['Lon']

from folium.plugins import HeatMap

burst = df_flight[df_flight['altitude'] == df_flight['altitude'].max()]
burst2 = df_other[df_other['altitude'] == df_other['altitude'].max()]
burst3 = df_model[df_model['Altitude'] == df_model['Altitude'].max()]


loc = [df_flight['lat'].iloc[0], df_flight['lng'].iloc[0]]
m = folium.Map(location=[burst['lat'], burst['lng']])
folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('stamenterrain').add_to(m)
folium.LayerControl().add_to(m)

folium.Marker(
    location=loc,
    popup='Start',
    icon=folium.Icon(icon='home')
).add_to(m)

folium.Marker(
    location=[burst['lat'], burst['lng']],
    popup='Burst '+str(burst['altitude'])+' m',
    icon=folium.Icon(icon='fire', color='black')
).add_to(m)

folium.Marker(
    location=[burst2['lat'], burst2['lon']],
    popup='Burst '+str(burst2['altitude'])+' m',
    icon=folium.Icon(icon='fire', color='blue')
).add_to(m)

folium.Marker(
    location=[burst3['Lat'], burst3['Lon']],
    popup='Burst '+str(burst3['Altitude'])+' m',
    icon=folium.Icon(icon='fire', color='red')
).add_to(m)

folium.Marker(
    location=[df_flight['lat'].iloc[-1],df_flight['lng'].iloc[-1]],
    popup='Land '+str(df_flight['lat'].iloc[-1]) + ' , ' + str(df_flight['lng'].iloc[-1]),
    icon=folium.Icon(icon='search', color='black')
).add_to(m)


folium.Marker(
    location=[df_other['lat'].iloc[-1],df_other['lon'].iloc[-1]],
    popup='Land '+str(df_other['lat'].iloc[-1]) + ' , ' + str(df_other['lon'].iloc[-1]),
    icon=folium.Icon(icon='search', color='blue')
).add_to(m)

folium.Marker(
    location=[df_model['Lat'].iloc[-1],df_model['Lon'].iloc[-1]],
    popup='Land '+str(df_model['Lat'].iloc[-1]) + ' , ' + str(df_model['Lon'].iloc[-1]),
    icon=folium.Icon(icon='search', color='red')
).add_to(m)

# folium.PolyLine(np.dstack((df_model6['Lat'],df_model6['Lon'])), color="red", weight=3, opacity=0.9).add_to(m)
# folium.PolyLine(np.dstack((df_model2['Lat'],df_model2['Lon'])), color="green", weight=3, opacity=0.9).add_to(m)

folium.PolyLine(np.dstack((model_lat,model_lon)), color="red", weight=3, opacity=0.9).add_to(m)
folium.PolyLine(np.dstack((f_lat,f_lon)), color="black", weight=3, opacity=0.9).add_to(m)
folium.PolyLine(np.dstack((other_lat,other_lon)), color="blue", weight=3, opacity=0.9).add_to(m)

m

In [152]:
df_flight = pd.read_csv('Flights/R0150226_edit.csv')
f_lat = df_flight['lat']
f_lon = df_flight['lng']

df_other = pd.read_csv('OtherModels/R0150226.csv')
other_lat = df_other['lat'] 
other_lon = df_other['lon']

df_model = pd.read_csv('Predictions/R0150226_p4.csv')
model_lat = df_model['Lat'] 
model_lon = df_model['Lon']

from folium.plugins import HeatMap

burst = df_flight[df_flight['altitude'] == df_flight['altitude'].max()]
burst2 = df_other[df_other['altitude'] == df_other['altitude'].max()]
burst3 = df_model[df_model['Altitude'] == df_model['Altitude'].max()]


loc = [df_flight['lat'].iloc[0], df_flight['lng'].iloc[0]]
m = folium.Map(location=[burst['lat'], burst['lng']])
folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('stamenterrain').add_to(m)
folium.LayerControl().add_to(m)

folium.Marker(
    location=loc,
    popup='Start',
    icon=folium.Icon(icon='home')
).add_to(m)

folium.Marker(
    location=[burst['lat'], burst['lng']],
    popup='Burst '+str(burst['altitude'])+' m',
    icon=folium.Icon(icon='fire', color='black')
).add_to(m)

folium.Marker(
    location=[burst2['lat'], burst2['lon']],
    popup='Burst '+str(burst2['altitude'])+' m',
    icon=folium.Icon(icon='fire', color='blue')
).add_to(m)

folium.Marker(
    location=[burst3['Lat'], burst3['Lon']],
    popup='Burst '+str(burst3['Altitude'])+' m',
    icon=folium.Icon(icon='fire', color='red')
).add_to(m)

folium.Marker(
    location=[df_flight['lat'].iloc[-1],df_flight['lng'].iloc[-1]],
    popup='Land '+str(df_flight['lat'].iloc[-1]) + ' , ' + str(df_flight['lng'].iloc[-1]),
    icon=folium.Icon(icon='search', color='black')
).add_to(m)


folium.Marker(
    location=[df_other['lat'].iloc[-1],df_other['lon'].iloc[-1]],
    popup='Land '+str(df_other['lat'].iloc[-1]) + ' , ' + str(df_other['lon'].iloc[-1]),
    icon=folium.Icon(icon='search', color='blue')
).add_to(m)

folium.Marker(
    location=[df_model['Lat'].iloc[-1],df_model['Lon'].iloc[-1]],
    popup='Land '+str(df_model['Lat'].iloc[-1]) + ' , ' + str(df_model['Lon'].iloc[-1]),
    icon=folium.Icon(icon='search', color='red')
).add_to(m)

# folium.PolyLine(np.dstack((df_model6['Lat'],df_model6['Lon'])), color="red", weight=3, opacity=0.9).add_to(m)
# folium.PolyLine(np.dstack((df_model2['Lat'],df_model2['Lon'])), color="green", weight=3, opacity=0.9).add_to(m)

folium.PolyLine(np.dstack((model_lat,model_lon)), color="red", weight=3, opacity=0.9).add_to(m)
folium.PolyLine(np.dstack((f_lat,f_lon)), color="black", weight=3, opacity=0.9).add_to(m)
folium.PolyLine(np.dstack((other_lat,other_lon)), color="blue", weight=3, opacity=0.9).add_to(m)

m

Sensibility analysis

In [428]:
count = 1
CD = [0, 0.6, 0.7, 0.8, 0.9]
while(count<5):

    mass_min = 3.0
    mass_max = 3.0
    
    time_step = 300

    Ap_min = 2.0
    Ap_max = 2.0

    burst_altitude_min = 30000
    burst_altitude_max = 30000

    ascent_rate_min = 5.0
    ascent_rate_max = 5.0
    Cd = CD[count]

    mass = round(random.uniform(mass_min, mass_max), 2)
    Ap = round(random.uniform(Ap_min, Ap_max), 2)
    burst_altitude = round(random.uniform(burst_altitude_min, burst_altitude_max), 2)
    ascent_rate = round(random.uniform(ascent_rate_min, ascent_rate_max), 2)



    df = simulate(mass=mass, Cd=Cd, Ap=Ap, 
                 start_location = [11.89705, 49.42436], 
                 start_altitude=0, burst_altitude=burst_altitude, 
                 time_step=300, ascent_rate=ascent_rate)

    df.to_csv('Predictions/Cd_'+str(count)+'.csv')

    count +=1
    print(count)
print('finish')

0 meter
1500.0 meter
3000.0 meter
4500.0 meter
6000.0 meter
7500.0 meter
9000.0 meter
10500.0 meter
12000.0 meter
13500.0 meter
15000.0 meter
16500.0 meter
18000.0 meter
19500.0 meter
21000.0 meter
22500.0 meter
24000.0 meter
25500.0 meter
27000.0 meter
28500.0 meter
30000.0 meter
28475.0 meter
26638.0 meter
24859.0 meter
23113.0 meter
21333.0 meter
19502.0 meter
17595.0 meter
15570.0 meter
13389.0 meter
10992.0 meter
8300.0 meter
5188.0 meter
1447.0 meter
0 meter
2
0 meter
1500.0 meter
3000.0 meter
4500.0 meter
6000.0 meter
7500.0 meter
9000.0 meter
10500.0 meter
12000.0 meter
13500.0 meter
15000.0 meter
16500.0 meter
18000.0 meter
19500.0 meter
21000.0 meter
22500.0 meter
24000.0 meter
25500.0 meter
27000.0 meter
28500.0 meter
30000.0 meter
28588.0 meter
26911.0 meter
25285.0 meter
23667.0 meter
22026.0 meter
20358.0 meter
18630.0 meter
16821.0 meter
14893.0 meter
12817.0 meter
10537.0 meter
7985.0 meter
5050.0 meter
1558.0 meter
0 meter
3
0 meter
1500.0 meter
3000.0 meter
4500.0 met

In [427]:
m1 = pd.read_csv('Predictions/Time_Step_1.csv')
m1_lat = m1['Lat']
m1_lon = m1['Lon']

m2 = pd.read_csv('Predictions/Time_Step_2.csv')
m2_lat = m2['Lat']
m2_lon = m2['Lon']

m3 = pd.read_csv('Predictions/Time_Step_3.csv')
m3_lat = m3['Lat']
m3_lon = m3['Lon']

m4 = pd.read_csv('Predictions/Time_Step_4.csv')
m4_lat = m4['Lat']
m4_lon = m4['Lon']

from folium.plugins import HeatMap

burst1 = m1[m1['Altitude'] == m1['Altitude'].max()]
burst2 = m2[m2['Altitude'] == m2['Altitude'].max()]
burst3 = m3[m3['Altitude'] == m3['Altitude'].max()]
burst4 = m4[m4['Altitude'] == m4['Altitude'].max()]

loc = [m1['Lat'].iloc[0], m1['Lon'].iloc[0]]
m = folium.Map(location=[burst1['Lat'], burst1['Lon']])
folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('stamenterrain').add_to(m)
folium.LayerControl().add_to(m)

folium.Marker(
    location=loc,
    popup='Start',
    icon=folium.Icon(icon='home')
).add_to(m)

folium.Marker(
    location=[burst1['Lat'], burst1['Lon']],
    popup='Burst '+str(burst1['Altitude'])+' m',
    icon=folium.Icon(icon='fire', color='black')
).add_to(m)

folium.Marker(
    location=[burst2['Lat'], burst2['Lon']],
    popup='Burst '+str(burst2['Altitude'])+' m',
    icon=folium.Icon(icon='fire', color='blue')
).add_to(m)

folium.Marker(
    location=[burst3['Lat'], burst3['Lon']],
    popup='Burst '+str(burst3['Altitude'])+' m',
    icon=folium.Icon(icon='fire', color='red')
).add_to(m)

folium.Marker(
    location=[burst4['Lat'], burst4['Lon']],
    popup='Burst '+str(burst4['Altitude'])+' m',
    icon=folium.Icon(icon='fire', color='red')
).add_to(m)

folium.Marker(
    location=[m1['Lat'].iloc[-1],m1['Lon'].iloc[-1]],
    popup='Land '+str(m1['Lat'].iloc[-1]) + ' , ' + str(m1['Lon'].iloc[-1]),
    icon=folium.Icon(icon='search', color='black')
).add_to(m)

folium.Marker(
    location=[m2['Lat'].iloc[-1],m2['Lon'].iloc[-1]],
    popup='Land '+str(m2['Lat'].iloc[-1]) + ' , ' + str(m2['Lon'].iloc[-1]),
    icon=folium.Icon(icon='search', color='black')
).add_to(m)

folium.Marker(
    location=[m3['Lat'].iloc[-1],m3['Lon'].iloc[-1]],
    popup='Land '+str(m3['Lat'].iloc[-1]) + ' , ' + str(m3['Lon'].iloc[-1]),
    icon=folium.Icon(icon='search', color='black')
).add_to(m)

folium.Marker(
    location=[m4['Lat'].iloc[-1],m4['Lon'].iloc[-1]],
    popup='Land '+str(m4['Lat'].iloc[-1]) + ' , ' + str(m4['Lon'].iloc[-1]),
    icon=folium.Icon(icon='search', color='black')
).add_to(m)

# folium.PolyLine(np.dstack((df_model6['Lat'],df_model6['Lon'])), color="red", weight=3, opacity=0.9).add_to(m)
# folium.PolyLine(np.dstack((df_model2['Lat'],df_model2['Lon'])), color="green", weight=3, opacity=0.9).add_to(m)

folium.PolyLine(np.dstack((m1_lat,m1_lon)), color="red", weight=3, opacity=1).add_to(m)
folium.PolyLine(np.dstack((m2_lat,m2_lon)), color="green", weight=3, opacity=1).add_to(m)
folium.PolyLine(np.dstack((m3_lat,m3_lon)), color="blue", weight=3, opacity=1).add_to(m)
folium.PolyLine(np.dstack((m4_lat,m4_lon)), color="black", weight=3, opacity=1).add_to(m)


m